In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine

In [2]:
# Read database created in convert_to_sql.py
engine = create_engine('sqlite:///../data/online_retail_sql.db')
df = pd.read_sql_table('online_retail_sql', con=engine)

In [3]:
# Quick overview of dataframe and summary statistics
print(f'''{df.describe()},
      {df.info()},
      {df.columns}''')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  object        
 7   Country      397884 non-null  object        
 8   Revenue      397884 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.3+ MB
            Quantity                    InvoiceDate      UnitPrice  \
count  397884.000000                         397884  397884.000000   
mean       12.988238  2011-07-10 23:41:23.511023360       3.116488   
min         1.000000            2010-12-01 08:26

In [4]:
# Display first few rows to understand data layout
df.head(20)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08


In [5]:
# Calculate and display the top selling products
# Additionally shown in SQL query
product_revenue = df.groupby('Description')['Revenue'].sum().sort_values(ascending=False)
product_revenue.head(20)

Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142592.95
WHITE HANGING HEART T-LIGHT HOLDER    100448.15
JUMBO BAG RED RETROSPOT                85220.78
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
POSTAGE                                77803.96
PARTY BUNTING                          68844.33
ASSORTED COLOUR BIRD ORNAMENT          56580.34
Manual                                 53779.93
RABBIT NIGHT LIGHT                     51346.20
CHILLI LIGHTS                          46286.51
PAPER CHAIN KIT 50'S CHRISTMAS         42660.83
PICNIC BASKET WICKER 60 PIECES         39619.50
BLACK RECORD COVER FRAME               39064.55
JUMBO BAG PINK POLKADOT                37289.59
DOORMAT KEEP CALM AND COME IN          35913.85
SPOTTY BUNTING                         35539.25
WOOD BLACK BOARD ANT WHITE FINISH      34478.01
SET OF 3 CAKE TINS PANTRY DESIGN       33347.80
JAM MAKING SET WITH JARS               32662.97
Name: Revenue, dtype: float6

In [6]:
# Calculate and display the top customers and their spendings
# Additionally shown in SQL query
customer_spendings = df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False)
customer_spendings.head(20)

CustomerID
14646.0    280206.02
18102.0    259657.30
17450.0    194550.79
16446.0    168472.50
14911.0    143825.06
12415.0    124914.53
14156.0    117379.63
17511.0     91062.38
16029.0     81024.84
12346.0     77183.60
16684.0     66653.56
14096.0     65164.79
13694.0     65039.62
15311.0     60767.90
13089.0     58825.83
17949.0     58510.48
15769.0     56252.72
15061.0     54534.14
14298.0     51527.30
14088.0     50491.81
Name: Revenue, dtype: float64

In [7]:
# Display a bar plot using plotly showcasing the top 20 selling products
product_revenue_fig = px.bar(
    x = product_revenue.head(20).index,
    y = product_revenue.head(20).round(-3).values,
    color = product_revenue.head(20).round(-3).values,
    labels = {'x': 'Product', 'y': 'Revenue'},
    title = 'Top Selling Products'
)

# Hide x axis tick labels, which were too crowded
product_revenue_fig.update_layout(xaxis=dict(showticklabels=False))
product_revenue_fig.show()

In [8]:
# Display a bar plot using plotly showcasing the top 20 customers
customer_spendings_fig = px.bar(
    x = customer_spendings.head(20).index,
    y = customer_spendings.head(20).values,
    color = customer_spendings.head(20).values,
    labels = {'x': 'CustomerID', 'y': 'Spending'},
    title = 'Top Customers'
)

# Hide cluttered x axis tick labels
customer_spendings_fig.update_layout(xaxis=dict(showticklabels=False))
customer_spendings_fig.show()

In [9]:
df['InvoiceDate'] = df['InvoiceDate'].dt.date
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


In [10]:
revenue_over_time = df.groupby('InvoiceDate')['Revenue'].sum().sort_index(ascending=True)
revenue_over_time.head(20)

InvoiceDate
2010-12-01    46376.49
2010-12-02    47316.53
2010-12-03    23921.71
2010-12-05    31771.60
2010-12-06    31215.64
2010-12-07    53795.31
2010-12-08    39248.82
2010-12-09    38231.90
2010-12-10    33650.28
2010-12-12    17305.77
2010-12-13    27642.68
2010-12-14    29322.30
2010-12-15    30311.72
2010-12-16    49030.08
2010-12-17    20046.56
2010-12-19     7417.39
2010-12-20    19621.96
2010-12-21    15951.66
2010-12-22     4886.52
2010-12-23     5648.97
Name: Revenue, dtype: float64

In [11]:
# Create a line plot showcasing the revenue over time
revenue_over_time_fig = px.line(
    x=revenue_over_time.index,
    y=revenue_over_time.values.round(-1),
    labels = {'x': 'Date', 'y': 'Revenue'},
    title = 'Revenue Over Time',
    height = 800
)

# Add a range slider and selector
revenue_over_time_fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count = 1,
                 label = '1m',
                step = 'month',
                stepmode = 'backward'),
            dict(count = 6,
                label = '6m',
                step = 'month',
                stepmode = 'backward'),
            dict(count = 1,
                label = 'YTD',
                step = 'year',
                stepmode = 'todate'),
            dict(count = 1,
                label = '1y',
                step = 'year',
                stepmode = 'backward'),
            dict(step = 'all')
        ])
    ),
    rangeslider_visible=True
)

# Center title and increase font size
revenue_over_time_fig.update_layout(
    title_x = 0.5,
    title_xanchor = 'center',
    title_font = dict(
        size = 24
    )
)

# Display plot
revenue_over_time_fig.show()

In [16]:
# Display a bar plot using plotly showcasing the top countries
top_countries = pd.read_csv('../outputs/tables/top_countries_output.csv')
top_countries_fig = px.bar(
    x = top_countries['Country'].head(5),
    y = top_countries['TotalRevenue'].head(5),
    color = top_countries['TotalRevenue'].head(5),
    labels = {'x': 'CustomerID', 'y': 'Spending'},
    title = 'Top Customers'
)

# Hide cluttered x axis tick labels
top_countries_fig.update_layout(xaxis=dict(showticklabels=False))
top_countries_fig.show()

In [19]:
# Save graphs as pngs
product_revenue_fig.write_image('../outputs/graphs/top_selling_products.png', width = 1600, height = 800, scale = 2)
customer_spendings_fig.write_image('../outputs/graphs/top_customers.png', width = 1600, height = 800, scale = 2)
revenue_over_time_fig.write_image('../outputs/graphs/revenue_over_time.png', width = 1600, height = 800, scale = 2)
top_countries_fig.write_image('../outputs/graphs/top_countries.png', width = 1600, height = 800, scale = 2)